In [29]:
# import packages 

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [30]:
# import training and test data 

save_location_path = "location on your local drive"

train = pd.read_csv("/Users/emmanuelomoregie/Documents/Kaggle Projects/Space_Titanic/spaceship-titanic/train.csv")

test = pd.read_csv("/Users/emmanuelomoregie/Documents/Kaggle Projects/Space_Titanic/spaceship-titanic/test.csv")
 

In [31]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [32]:
train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [33]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [34]:
test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.000000,4195.000000,4171.000000,4179.000000,4176.000000,4197.000000
mean,28.658146,219.266269,439.484296,177.295525,303.052443,310.710031
std,14.179072,607.011289,1527.663045,560.821123,1117.186015,1246.994742
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,53.000000,78.000000,33.000000,50.000000,36.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


## New Variables Via Feature Engineering

1. Deck - The Deck which the passenger was on derived from the cabin variable
2. Num - The number related to where the passenger stayed derived from th cabin variable 
3. Side - The side of the spaceship for which the passenger was on derived from the cabin varibale 
4. Total_Spend - Total amount of money spent by passengers on the spaceship 
5. PassengerId_end - last three digits from the passengerId variable 

In [35]:
train['Deck'] = [i.split('/')[0] if type(i) != float else i for i in train['Cabin']]
train['Num'] = [i.split('/')[1] if type(i) != float else i for i in train['Cabin']]
train['Side'] = [i.split('/')[2] if type(i) != float else i for i in train['Cabin']]
train['TotalSpend'] = train['RoomService'] + train['FoodCourt'] + train['ShoppingMall'] + train['Spa'] + train['VRDeck']
train['PassengerId_end'] = [i.split('_')[1] if type(i) != float else i for i in train['PassengerId']]

In [36]:
test['Deck'] = [i.split('/')[0] if type(i) != float else i for i in test['Cabin']]
test['Num'] = [i.split('/')[1] if type(i) != float else i for i in test['Cabin']]
test['Side'] = [i.split('/')[2] if type(i) != float else i for i in test['Cabin']]
test['TotalSpend'] = test['RoomService'] + test['FoodCourt'] + test['ShoppingMall'] + test['Spa'] + test['VRDeck']
test['PassengerId_end'] = [i.split('_')[1] if type(i) != float else i for i in test['PassengerId']]

## Dealing with Missing Values 

In [37]:
train.isnull().sum()

PassengerId          0
HomePlanet         201
CryoSleep          217
Cabin              199
Destination        182
Age                179
VIP                203
RoomService        181
FoodCourt          183
ShoppingMall       208
Spa                183
VRDeck             188
Name               200
Transported          0
Deck               199
Num                199
Side               199
TotalSpend         908
PassengerId_end      0
dtype: int64

In [38]:
missing_values = list(train.isnull().sum())
column_names = list(train.columns)

In [39]:
percent_of_missing = {column_names[i]:(str(missing_values[i]/len(train)*100)) + '%' for i in range(len(missing_values))}

In [40]:
((sum(missing_values)/(len(train) * len(missing_values))) * 100)

2.3182597007876877

From my dataset about 2% is missing data, as I will still have 98% of my data remaining.

We are going to create 4 types of dataset:

1. Drop missing data 
2. Impute Missing data using mode (for cateorgical variables) and medium/mean (for numerical variables)

In the dataset, The variables that are categorical variables is as follows:

1. HomePlanet
2. CryoSleep
3. Destination
4. VIP
5. Deck
6. Side 
7. Passengerid_end

The variables that are numerical variables is as follows:

1. Age
2. RoomService
3. FoodCourt
4. ShoppingMall
5. Spa
6. VRDeck
7. TotalSpend


In [52]:
def model_performance(model_list, X, y):
    
    result = dict()
    for model_name, model in model_list.items():
        model.fit(X, y)
        average_accuracy = cross_val_score(model, X, y, cv=5)
        result[model_name] = np.mean(average_accuracy)
    
    return result

In [59]:
cat_columns = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side', 'PassengerId', 'PassengerId_end']
num_columns = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalSpend']

In [60]:
encoder = LabelEncoder()

## Dropped Dataset 

In [42]:
# dataset with dropped data

train_dropped = train.dropna()

#### Dropped Dataset Preprocessing 

In [44]:
for i in cat_columns:
    train_dropped[i] = encoder.fit_transform(train_dropped[i])

In [45]:
train_dropped =  train_dropped.drop(['Cabin', 'Name', 'Num'], axis = 1)

In [46]:
X_dropped = train_dropped.drop(['PassengerId', 'Transported'], axis=1)
y_dropped = train_dropped['Transported']

In [47]:
X_dropped_train, X_dropped_test, y_dropped_train, y_dropped_test = train_test_split(X_dropped, y_dropped,test_size=0.2, random_state=42)


#### Classifers 

In [48]:
tree_clf_dropped = DecisionTreeClassifier(criterion='gini', max_depth=5, min_samples_leaf=4,min_samples_split=2)
rand_forest_clf_dropped = RandomForestClassifier(max_depth=10, min_samples_split=2, n_estimators=200)
ada_clf_dropped = AdaBoostClassifier(learning_rate=0.5, n_estimators=100)
GB_clf_dropped = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, n_estimators=50)
xgb_clf_dropped = xgb.XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=300)

In [49]:
dropped_models = {'Decision_Tree_Dropped':tree_clf_dropped, 
                  'Random_forest_Dropped': rand_forest_clf_dropped,
                  'Ada_Boost_Dropped': ada_clf_dropped,
                  'Gradient_Boost_Dropped': GB_clf_dropped,
                 'Xtreme_Graident_Boost_Dropped':xgb_clf_dropped}

In [54]:
dropped_model_performance = model_performance(dropped_models, X_dropped, y_dropped)

In [55]:
print(dropped_model_performance)

{'Decision_Tree': 0.7838353102048716, 'Random_forest': 0.7982169790684864, 'Ada_Boost': 0.7932213366988059, 'Gradient_Boost': 0.8013947852736145, 'Xterme_Graident_Boost': 0.8054816813467081}


## Imputed Dataset - with Mode and Median

In [72]:
train_impute = train.copy()

In [73]:
# Catergorical Variable

train_impute['HomePlanet'] = train_impute['HomePlanet'].fillna(train_impute['HomePlanet'].mode()[0])
train_impute['CryoSleep'] = train_impute['CryoSleep'].fillna(train_impute['CryoSleep'].mode()[0])
train_impute['Destination'] = train_impute['Destination'].fillna(train_impute['Destination'].mode()[0])
train_impute['VIP'] = train_impute['VIP'].fillna(train_impute['VIP'].mode()[0])
train_impute['Deck'] = train_impute['Deck'].fillna(train_impute['Deck'].mode()[0])
train_impute['Side'] = train_impute['Side'].fillna(train_impute['Side'].mode()[0])
train_impute['PassengerId_end'] = train_impute['PassengerId_end'].fillna(train_impute['PassengerId_end'].mode()[0])

# Numerical Variable

train_impute['Age'] = train_impute['Age'].fillna(np.median(train['Age'].dropna()))
train_impute['RoomService'] = train_impute['RoomService'].fillna(np.median(train['RoomService'].dropna()))
train_impute['FoodCourt'] = train_impute['FoodCourt'].fillna(np.median(train['FoodCourt'].dropna()))
train_impute['ShoppingMall'] = train_impute['ShoppingMall'].fillna(np.median(train['ShoppingMall'].dropna()))
train_impute['Spa'] = train_impute['Spa'].fillna(np.median(train['Spa'].dropna()))
train_impute['VRDeck'] = train_impute['VRDeck'].fillna(np.median(train['VRDeck'].dropna()))
train_impute['TotalSpend'] = train_impute['TotalSpend'].fillna(np.median(train['TotalSpend'].dropna()))

In [74]:
for i in cat_columns:
    train_impute[i] = encoder.fit_transform(train_impute[i])

In [75]:
train_impute =  train_impute.drop(['Cabin', 'Name', 'Num'], axis = 1)

In [76]:
X_impute = train_impute.drop(['PassengerId', 'Transported'], axis=1)
y_impute = train_impute['Transported']

In [77]:
X_impute_train, X_impute_test, y_impute_train, y_impute_test = train_test_split(X_impute, y_impute,test_size=0.2, random_state=42)


In [78]:
tree_clf_impute = DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_leaf=4, min_samples_split=5)
rand_forest_clf_impute = RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=200)
ada_clf_impute = AdaBoostClassifier(learning_rate=0.5, n_estimators=150)
GB_clf_impute = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, n_estimators=150)
xgb_clf_impute = xgb.XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=300)

In [79]:
impute_models = {'Decision_Tree_Impute':tree_clf_impute, 
                  'Random_forest_Impute': rand_forest_clf_impute,
                  'Ada_Boost_Impute': ada_clf_impute,
                  'Gradient_Boost_Impute': GB_clf_impute,
                 'Xtreme_Graident_Boost_Impute':xgb_clf_impute}

In [80]:
impute_model_performance = model_performance(impute_models, X_impute, y_impute)

In [81]:
print(impute_model_performance)

{'Decision_Tree_Impute': 0.7698181103513719, 'Random_forest_Impute': 0.7970808455053001, 'Ada_Boost_Impute': 0.7904073674340305, 'Gradient_Boost_Impute': 0.7969656383607366, 'Xterme_Graident_Boost_Impute': 0.796965042803987}


# Submission 

Although in this case Dropped Data with Xtreme Gradient Boost has the best score, for my submission Impute data with Xtreme Gradient Boost is used. From my submission research these were my results 

- Dropped Dataset 

1. Decision Tree: 0.7804718184685072
2. Random Forest: 0.8009102376652045
3. Ada Boost: 0.7976905406955075
4. Gradient Boost: 0.8048835684183366
5. XGB: 0.8062075341876666

- Impute Dataset 

1. Decision Tree: 0.7825709985570134
2. Random Forest: 0.8061551908724638
3. Ada Boost: 0.7993986004582387
4. Gradient Boost: 0.8031355734966305
5. XGB: 0.8068744084531081 - The Best

In [82]:
test_impute = test.copy()

In [84]:

# Catergorical Variable

test_impute['HomePlanet'] = test_impute['HomePlanet'].fillna(test['HomePlanet'].mode()[0])
test_impute['CryoSleep'] = test_impute['CryoSleep'].fillna(test['CryoSleep'].mode()[0])
test_impute['Destination'] = test_impute['Destination'].fillna(test['Destination'].mode()[0])
test_impute['VIP'] = test_impute['VIP'].fillna(test['VIP'].mode()[0])
test_impute['Deck'] = test_impute['Deck'].fillna(test['Deck'].mode()[0])
test_impute['Side'] = test_impute['Side'].fillna(test['Side'].mode()[0])
test_impute['PassengerId_end'] = test_impute['PassengerId_end'].fillna(test['PassengerId_end'].mode()[0])

# Numerical Variable

test_impute['Age'] = test_impute['Age'].fillna(np.median(test['Age'].dropna()))
test_impute['RoomService'] = test_impute['RoomService'].fillna(np.median(test['RoomService'].dropna()))
test_impute['FoodCourt'] = test_impute['FoodCourt'].fillna(np.median(test['FoodCourt'].dropna()))
test_impute['ShoppingMall'] = test_impute['ShoppingMall'].fillna(np.median(test['ShoppingMall'].dropna()))
test_impute['Spa'] = test_impute['Spa'].fillna(np.median(test['Spa'].dropna()))
test_impute['VRDeck'] = test_impute['VRDeck'].fillna(np.median(test['VRDeck'].dropna()))
test_impute['TotalSpend'] = test_impute['TotalSpend'].fillna(np.median(test['TotalSpend'].dropna()))

In [85]:
for i in cat_columns:
    test_impute[i] = encoder.fit_transform(test_impute[i])

In [86]:
test_impute =  test_impute.drop(['Cabin', 'Name', 'Num'], axis = 1)

In [94]:
X_train = train_impute.drop(['PassengerId', 'Transported'], axis=1)
y_train = train_impute['Transported']

In [95]:
xgb_clf = xgb.XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=300)

In [96]:
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [98]:
X_test = test_impute.drop(['PassengerId'], axis = 1)

In [99]:
y_pred = xgb_clf.predict(X_test)

In [100]:
submission = test[['PassengerId']]

In [101]:
submission['Transported'] = y_pred

In [102]:
submission['Transported'] = submission['Transported'].astype('bool')

In [104]:
submission.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [ ]:
save_location_path = "location on your local drive"

submission.to_csv(save_location_path, index=False)

After Submission I received a score of 0.80079, which placed me in the top 30%